# ADM1 Model Tutorial

This notebook demonstrates how to use the **ADM1Model** (Anaerobic Digestion Model No. 1) from OpenAD-lib for simulating biogas production from anaerobic digestion processes.

## Overview

ADM1 is a comprehensive mechanistic model that simulates:
- Biochemical processes (disintegration, hydrolysis, uptake, decay)
- Physicochemical processes (acid-base equilibria, gas-liquid transfer)
- 38 state variables covering soluble, particulate, ion, and gas phase components

## 1. Setup and Imports

In [ ]:
import sys
import os

# Add library to path if not installed
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'src'))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import ADM1 Model
from openad_lib.models.mechanistic import ADM1Model
from openad_lib.models.mechanistic.adm1_model import ADM1Parameters

print("Imports successful!")

## 2. Understanding ADM1 Parameters

The ADM1 model uses a comprehensive set of parameters based on the BSM2 (Benchmark Simulation Model No. 2) report by Rosen et al. (2006).

In [ ]:
# Create default parameters
params = ADM1Parameters()

print("=== Physical Parameters ===")
print(f"Liquid Volume (V_liq): {params.V_liq} m³")
print(f"Gas Headspace (V_gas): {params.V_gas} m³")
print(f"Operating Temperature: {params.T_op - 273.15}°C ({params.T_op} K)")

print("\n=== Key Kinetic Parameters ===")
print(f"Disintegration rate (k_dis): {params.k_dis} d⁻¹")
print(f"Hydrolysis rate (k_hyd_ch): {params.k_hyd_ch} d⁻¹")
print(f"Max uptake sugars (k_m_su): {params.k_m_su} d⁻¹")
print(f"Max uptake acetate (k_m_ac): {params.k_m_ac} d⁻¹")

In [ ]:
# Customize parameters for your reactor
custom_params = ADM1Parameters(
    V_liq=5000,      # 5000 m³ reactor
    V_gas=300,       # 300 m³ headspace
    T_op=308.15,     # 35°C mesophilic
    k_dis=0.4,       # Slower disintegration
)

print(f"Custom reactor volume: {custom_params.V_liq} m³")
print(f"Custom temperature: {custom_params.T_op - 273.15}°C")

## 3. Initialize the ADM1 Model

In [ ]:
# Initialize with default parameters
model = ADM1Model()

# Or with custom parameters
# model = ADM1Model(params=custom_params)

print(f"Model initialized with {len(model.STATE_NAMES)} state variables")
print(f"\nState variables:")
for i, name in enumerate(model.STATE_NAMES):
    print(f"  {i+1:2d}. {name}")

## 4. Load Data

The ADM1 model requires:
1. **Influent data**: Time series of input concentrations
2. **Initial state**: Starting conditions for the reactor

In [ ]:
# Define paths to sample data
DATA_DIR = os.path.join(os.path.dirname(os.getcwd()), 'src', 'openad_lib', 'data')

influent_path = os.path.join(DATA_DIR, 'sample_ADM1_influent_data.csv')
initial_path = os.path.join(DATA_DIR, 'sample_initial_state.csv')

print(f"Influent data: {os.path.basename(influent_path)}")
print(f"Initial state: {os.path.basename(initial_path)}")

In [ ]:
# Load the data
model.load_data(
    influent_path=influent_path,
    initial_state_path=initial_path
)

print("Data loaded successfully!")
print(f"\nInfluent data shape: {model.influent_data.shape}")
print(f"Simulation period: {model.influent_data['time'].min():.0f} to {model.influent_data['time'].max():.0f} days")

In [ ]:
# View the influent data
print("Influent data columns:")
print(model.influent_data.columns.tolist())

model.influent_data.head()

In [ ]:
# View initial state
print("Initial state values:")
for i, (name, value) in enumerate(zip(model.STATE_NAMES[:12], model.initial_state[:12])):
    print(f"  {name}: {value:.4f}")

## 5. Run Simulation

The simulation uses an ODE solver (BDF method for stiff systems) with a DAE solver for algebraic constraints.

In [ ]:
# Run the simulation
print("Starting ADM1 simulation...")
print("(This may take a few minutes for long simulations)\n")

results = model.run(
    solver_method="BDF",  # Best for stiff systems
    verbose=True
)

print(f"\nSimulation complete!")
print(f"Results shape: {results.shape}")

## 6. Analyze Results

In [ ]:
# Get results as DataFrames
states_df, gas_df = model.get_results()

print("State results:")
states_df.head()

In [ ]:
# Gas production results
print("Gas flow results:")
gas_df.head()

In [ ]:
# Summary statistics
print("=== Simulation Summary ===")
print(f"\nBiogas Production:")
print(f"  Mean: {gas_df['q_gas'].mean():.1f} m³/day")
print(f"  Max: {gas_df['q_gas'].max():.1f} m³/day")
print(f"  Total: {gas_df['q_gas'].sum():.0f} m³")

print(f"\nMethane Production:")
print(f"  Mean: {gas_df['q_ch4'].mean():.1f} m³/day")
print(f"  Total: {gas_df['q_ch4'].sum():.0f} m³")

print(f"\nReactor pH:")
print(f"  Mean: {states_df['pH'].mean():.2f}")
print(f"  Range: {states_df['pH'].min():.2f} - {states_df['pH'].max():.2f}")

## 7. Visualize Results

In [ ]:
# Use built-in plotting
model.plot_results()

In [ ]:
# Custom plots
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Biogas production
axes[0, 0].plot(gas_df['time'], gas_df['q_gas'], 'b-', linewidth=1.5)
axes[0, 0].set_xlabel('Time (days)')
axes[0, 0].set_ylabel('Biogas Flow (m³/day)')
axes[0, 0].set_title('Biogas Production')
axes[0, 0].grid(True, alpha=0.3)

# Methane production
axes[0, 1].plot(gas_df['time'], gas_df['q_ch4'], 'g-', linewidth=1.5)
axes[0, 1].set_xlabel('Time (days)')
axes[0, 1].set_ylabel('CH₄ Flow (m³/day)')
axes[0, 1].set_title('Methane Production')
axes[0, 1].grid(True, alpha=0.3)

# pH
axes[0, 2].plot(states_df['time'], states_df['pH'], 'r-', linewidth=1.5)
axes[0, 2].set_xlabel('Time (days)')
axes[0, 2].set_ylabel('pH')
axes[0, 2].set_title('Reactor pH')
axes[0, 2].axhline(y=7.0, color='gray', linestyle='--', alpha=0.5)
axes[0, 2].grid(True, alpha=0.3)

# VFA concentrations
axes[1, 0].plot(states_df['time'], states_df['S_ac'], label='Acetate')
axes[1, 0].plot(states_df['time'], states_df['S_pro'], label='Propionate')
axes[1, 0].plot(states_df['time'], states_df['S_bu'], label='Butyrate')
axes[1, 0].set_xlabel('Time (days)')
axes[1, 0].set_ylabel('Concentration (kg COD/m³)')
axes[1, 0].set_title('VFA Concentrations')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Biomass
axes[1, 1].plot(states_df['time'], states_df['X_ac'], label='Acetoclastic')
axes[1, 1].plot(states_df['time'], states_df['X_h2'], label='Hydrogenotrophic')
axes[1, 1].set_xlabel('Time (days)')
axes[1, 1].set_ylabel('Biomass (kg COD/m³)')
axes[1, 1].set_title('Methanogenic Biomass')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

# Inorganic nitrogen
axes[1, 2].plot(states_df['time'], states_df['S_IN'] * 1000, 'purple', linewidth=1.5)
axes[1, 2].set_xlabel('Time (days)')
axes[1, 2].set_ylabel('S_IN (mol/m³)')
axes[1, 2].set_title('Inorganic Nitrogen')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Save Results

In [ ]:
# Save to CSV
output_path = 'adm1_simulation_results.csv'
model.save_results(output_path)
print(f"Results saved to {output_path}")

## 9. Working with Feedstocks

You can use the FeedstockLibrary to calculate ADM1 inputs from practical feedstock measurements.

In [ ]:
from openad_lib.feedstock import FeedstockLibrary

# Load feedstock library
lib = FeedstockLibrary()

# Get feedstock properties
maize = lib.get("Maize")
print(f"Maize properties:")
print(f"  TS: {maize.ts} kg/m³")
print(f"  VS: {maize.vs} g/kg TS")
print(f"  BMP: {maize.bmp} NL CH4/kg VS")

# Calculate ADM1 inputs
adm1_inputs = lib.calculate_adm1_inputs(maize, flow_rate=130)

print(f"\nCalculated ADM1 inputs for Maize at 130 m³/day:")
for key, value in list(adm1_inputs.items())[:10]:
    print(f"  {key}: {value:.4f}")

## Summary

In this notebook, you learned how to:

1. **Initialize** the ADM1 model with default or custom parameters
2. **Load data** including influent time series and initial reactor state
3. **Run simulations** using appropriate ODE solvers
4. **Analyze results** including biogas production, pH, and VFA concentrations
5. **Visualize** model outputs with built-in and custom plots
6. **Integrate with feedstocks** for practical applications

### Next Steps

- Try different reactor parameters (temperature, volume)
- Experiment with different feedstock compositions
- Compare model predictions with measured data
- Use the LSTM or MTGP notebooks for data-driven approaches